In [1]:
using Pkg
Pkg.activate("../..")
Pkg.instantiate()

  Activating project at `~/dev/projects/AWID`
Precompiling project...
    508.2 ms  ✓ AWID
  1 dependency successfully precompiled in 2 seconds. 205 already precompiled.


In [2]:
using JLD2
X_train = load("../../data/embeddings/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../../data/embeddings/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../../data/embeddings/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../../data/embeddings/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../../data/embeddings/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../../data/embeddings/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings, 1)

50

In [3]:
using AWID.NeuralNetwork, AWID.AutoDiff

model = Chain(
    Embedding(length(vocab), embedding_dim),
    Permute((2,1,3)),
    Conv((3,), embedding_dim, 8, relu),
    MaxPool((8,)),
    Flatten(),
    Dense(128, 1, sigmoid)
)

Chain((Embedding(var ?
 ┣━ ^ 50×12837 Matrix{Float32}
 ┗━ ∇ Nothing), Permute((2, 1, 3)), Conv(var ?
 ┣━ ^ 3×50×8 Array{Float32, 3}
 ┗━ ∇ Nothing, var ?
 ┣━ ^ 8-element Vector{Float32}
 ┗━ ∇ Nothing, (3,), 50, 8, AWID.NeuralNetwork.Activations.relu), MaxPool((8,), (8,)), Flatten(), Dense(var ?
 ┣━ ^ 1×128 Matrix{Float32}
 ┗━ ∇ Nothing, var ?
 ┣━ ^ 1-element Vector{Float32}
 ┗━ ∇ Nothing, AWID.NeuralNetwork.Activations.sigmoid)))

In [4]:
using AWID.NeuralNetwork, AWID.AutoDiff, Printf

batch_size = 64

train_on_batch, test_loss_and_accuracy = setup_training_functions(
    model=model,
    loss_fn=binary_crossentropy,
    accuracy_fn=accuracy,
    optimizer=Adam(),
    x_test=X_test,
    y_test=y_test,
    batch_size=batch_size,
)

epochs = 5
for epoch in 1:epochs
    epoch_total_loss = 0.0f0
    epoch_total_acc = 0.0f0
    num_processed_batches = 0

    epoch_batches = get_epoch_batches(X_train, y_train, batch_size=batch_size, do_shuffle=true)

    t = @elapsed begin
        for (x_batch, y_batch) in epoch_batches
            batch_loss, batch_acc = train_on_batch(x_batch, y_batch)

            epoch_total_loss += batch_loss
            epoch_total_acc += batch_acc
            num_processed_batches += 1
        end

        train_loss = epoch_total_loss / num_processed_batches
        train_acc = epoch_total_acc / num_processed_batches

        test_loss, test_acc = test_loss_and_accuracy()
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)",
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Permute: (2, 1, 3)
Input: Matrix{Float32}(undef, 50, 0)


ArgumentError: ArgumentError: no valid permutation of dimensions